<a href="https://colab.research.google.com/github/spacexjedi/data-science-101/blob/master/rp_assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install mne

In [8]:
# -------------------------------
# importa bibliotecas
# -------------------------------
import mne
import scipy
import matplotlib
import numpy as np
import pandas as pd
import PIL
import json
from sklearn.svm import SVC
from scipy.signal import stft
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from google_drive_downloader import GoogleDriveDownloader as gdd

In [9]:
# ------------------------
# carrega dataset
# ------------------------
dataset_ids = {
    1: '1ym3JqI4_ZYNSBLxoF1vNxI5Hsgur_tha',
    2: '1tJ5tHbE-2jwr0gA33Gd873lRPao-e4dF',
    3: '1tXdpY-mser01POaP6Qwixny6LjsXXoXB',
    4: '1T00cliWp5yqqbbWZ7-rf2X4tOUQ-PvIQ',
    5: '1CYsBFMul9zB_vCy0XD7XVfKUl8vihDYZ',
    6: '1io6jdALBKeopELWXahPzuAi6EfYDgviW',
    7: '1YDkheRDPNDR1ujsqqC_SY6cebWHkw9Xt',
    8: '1jjoQJFDCi7O9Q-iaReAPpQnxC-HIKpQi',
}
label_id = '1mD5MXoh6tfQJFXIvdw2MQsEu6vZka6C0'
desc = '14kYNBZYdttqmSS_Vz6Bm_ztG9Uw1MC0y'

# escolhe qual dataset quer utilizar
DS = 1

# download do dataset
gdd.download_file_from_google_drive(file_id=dataset_ids[DS],
                                    dest_path='files/data.npy',
                                    showsize=True)
# download dos rótulos
gdd.download_file_from_google_drive(file_id=label_id,
                                    dest_path='files/labels.npy', showsize=True)

# download do arquivo de descrição
gdd.download_file_from_google_drive(file_id=desc,
                                    dest_path='files/descriptor.json',
                                    showsize=True)

data = np.load('files/data.npy') # x
labels = np.load('files/labels.npy') # y

desc_file = open('files/descriptor.json')
desc_json = json.loads(desc_file.read())
desc_file.close()

print('Estruturas => dados', data.shape, 'labels', labels.shape)
print(labels)
print(data)
print(desc_json)
s = pd.Series(labels)
print(s)
#tamanho = s.size
#print(tamanho)

Estruturas => dados (125, 257, 1205) labels (125,)
[4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1 5
 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1
 3 4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1
 5 4 3 2 4 1 2 5 3 4 1 3 1 3]
[[[-5.00655859e+03 -5.00834814e+03 -5.01312891e+03 ... -5.08810449e+03
   -5.09060254e+03 -5.08067334e+03]
  [ 1.89977979e+03  1.90244604e+03  1.89206934e+03 ...  1.89928076e+03
    1.90241650e+03  1.90704724e+03]
  [-3.43963203e+01 -3.03205070e+01 -3.97296295e+01 ... -1.38469672e+00
    1.05790544e+00  7.22508764e+00]
  ...
  [-9.77333594e+03 -9.76430273e+03 -9.76855176e+03 ... -9.53067578e+03
   -9.52037109e+03 -9.50803906e+03]
  [-2.43534985e+03 -2.42639746e+03 -2.43044531e+03 ... -2.41924609e+03
   -2.41576978e+03 -2.40342456e+03]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[-5.11064209e+03 -5.10834033e+03 -5.096208

In [11]:
# retirar eletrodo de referência
data = data[:,:256,:]
# dados / tempo
sampling_frequency = data.shape[-1] / 5
# montage - touca de eletro
montage = mne.channels.make_standard_montage('EGI_256')
ch_triais = data.shape[0]
ch_names = data.shape[1]
ch_ppf = data.shape[2]
ch_types = 'eeg'

# Criando as informações do obejto mne
info = mne.create_info(montage.ch_names, sampling_frequency, ch_types)
info.set_montage(montage)
events = np.array([[index, 0, event] for index, event in enumerate(labels)])
epoch = mne.EpochsArray(data, info, events)

# Escolha dos eletrodos, utilizando os mesmos das aula
filtered_epoch = epoch.copy().pick_channels(['E108', 'E109', 'E116', 'E125', 'E118', 'E117', 'E126',
                      'E139', 'E127', 'E138', 'E140', 'E150', 'E151'])
# Filtrando frequênica
filtered_epoch.filter(l_freq = 5.0, h_freq = 14.0)

'''
Obtendo as caracteristicas pelo predictive sparse decomposition.
'''
Y = np.load('files/labels.npy') # rótulos
X, _ = mne.time_frequency.psd_multitaper(filtered_epoch, fmin=5.0, fmax=14.0)
k_linear = []
k_polynomial = []
for f in [6.66, 7.5, 8.57, 10.0, 12]:
    X, _ = mne.time_frequency.psd_multitaper(filtered_epoch, fmin=f-0.5, fmax=f+0.5)
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])
    
    parameters_linear= {
        'kernel': ['linear'],
        'C':[0.01, 0.1, 1, 10, 100, 1000]
    }

    parameters_poly= {
        'kernel': ['poly'],
        'C':[0.01, 0.1, 1, 10, 100, 1000],
        'gamma': [10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]
    }
    # Stratified Shuffle Split cross validator
    ssc = StratifiedShuffleSplit(n_splits=50, test_size=0.2)

    grid_search_linear = GridSearchCV(SVC(), parameters_linear, n_jobs=-1, cv=ssc)
    grid_search_polynomial = GridSearchCV(SVC(), parameters_poly, n_jobs=-1, cv=ssc)

    grid_search_linear.fit(X, Y)
    grid_search_polynomial.fit(X, Y)

    k_linear.append(grid_search_linear.best_score_)
    k_polynomial.append(grid_search_polynomial.best_score_)

print(' Result -> linear value =', grid_search_linear.best_score_, grid_search_linear.best_params_)
print(' Result -> polynomial value =', grid_search_polynomial.best_score_, grid_search_polynomial.best_params_)


Not setting metadata
Not setting metadata
125 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Setting up band-pass filter from 5 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 399 samples (1.656 sec)

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectr

In [12]:
# Referências
## https://cs.nyu.edu/~yann/2010f-G22-2565-001/diglib/lecture12-sparse-coding.pdf
## https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html
## https://www.casadocodigo.com.br/pages/sumario-pandas-python
## material da aula 

SyntaxError: ignored